# Exemplar Simulations
Exemplar simulations investigating the performance degradation of Memristive Deep Neural Networks (MDNNs) when non-ideal device characteristics are accounted for using the CIFAR-10 dataset are provided below. Results can be plotted using `plot_all_exemplar.m`.

## 1. Define the MobileNetV2 Network Architecture

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Block(nn.Module):
    def __init__(self, in_planes, out_planes, expansion, stride):
        super(Block, self).__init__()
        self.stride = stride
        planes = expansion * in_planes
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(planes, affine=False)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, groups=planes)
        self.bn2 = nn.BatchNorm2d(planes, affine=False)
        self.conv3 = nn.Conv2d(planes, out_planes, kernel_size=1, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(out_planes, affine=False)

        self.shortcut = nn.Sequential()
        if stride == 1 and in_planes != out_planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0),
                nn.BatchNorm2d(out_planes, affine=False),
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out + self.shortcut(x) if self.stride==1 else out
        return out


class MobileNetV2(nn.Module):
    cfg = [(1,  16, 1, 1),
           (6,  24, 2, 1),
           (6,  32, 3, 2),
           (6,  64, 4, 2),
           (6,  96, 3, 1),
           (6, 160, 3, 2),
           (6, 320, 1, 1)]

    def __init__(self, num_classes=10):
        super(MobileNetV2, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32, affine=False)
        self.layers = self._make_layers(in_planes=32)
        self.conv2 = nn.Conv2d(320, 1280, kernel_size=1, stride=1, padding=0)
        self.bn2 = nn.BatchNorm2d(1280, affine=False)
        self.linear = nn.Linear(1280, num_classes)

    def _make_layers(self, in_planes):
        layers = []
        for expansion, out_planes, num_blocks, stride in self.cfg:
            strides = [stride] + [1]*(num_blocks-1)
            for stride in strides:
                layers.append(Block(in_planes, out_planes, expansion, stride))
                in_planes = out_planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layers(out)
        out = F.relu(self.bn2(self.conv2(out)))
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

## 2. Train MobileNetV2 Using CIFAR-10

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
import torchvision
import torchvision.transforms as transforms
from mobilenetv2 import MobileNetV2


def set_all_seeds(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True

def test(model, test_loader):
    correct = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        output = model(data.to(device))
        pred = output.data.max(1)[1]
        correct += pred.eq(target.to(device).data.view_as(pred)).cpu().sum()

    return 100. * float(correct) / float(len(test_loader.dataset))

set_all_seeds(0)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
epochs = 100
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])
train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=256, shuffle=True, num_workers=1)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False, num_workers=1)
model = MobileNetV2().to(device)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.1)
best_accuracy = 0
for epoch in range(0, epochs):
    print('Epoch: [%d]\t\t' % (epoch + 1), end='')
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.to(device))
        loss = criterion(output, target.to(device))
        loss.backward()
        optimizer.step()

    scheduler.step()
    model.eval()
    accuracy = test(model, test_loader)
    print('%2.2f%%' % accuracy)
    if accuracy > best_accuracy:
        print('Saving model...')
        torch.save(model.state_dict(), 'trained_model.pt')
        best_accuracy = accuracy

## 3. Figure 1 [A,E]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import memtorch
from memtorch.utils import LoadCIFAR10
import numpy as np
import pandas as pd
from mobilenetv2 import MobileNetV2
import torchvision
import copy
from memtorch.mn.Module import patch_model
from memtorch.map.Parameter import naive_map
from memtorch.bh.crossbar.Program import naive_program
from memtorch.bh.nonideality.NonIdeality import apply_nonidealities


def test(model, test_loader):
    correct = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        output = model(data.to(device))
        pred = output.data.max(1)[1]
        correct += pred.eq(target.to(device).data.view_as(pred)).cpu().sum()

    return 100. * float(correct) / float(len(test_loader.dataset))

device = torch.device('cuda')
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=256, shuffle=False, num_workers=1)
model = MobileNetV2().to(device)
try:
    model.load_state_dict(torch.load('trained_model.pt'), strict=False)
    model.eval()
except:
    raise Exception('trained_model.pt has not been found.')

print('Test Set Accuracy: \t%2.2f%%' % test(model, test_loader))

model = MobileNetV2().to(device)
model.load_state_dict(torch.load('trained_model.pt'), strict=True)
model.eval()
print(test(model, test_loader))
r_on = 1.4e4
r_off = 5e7

def trial(r_on, r_off, tile_shape, ADC_resolution, sigma):
    model_ = copy.deepcopy(model)
    reference_memristor = memtorch.bh.memristor.VTEAM
    if sigma == 0.:
        reference_memristor_params = {'time_series_resolution': 1e-10, 'r_off': r_off, 'r_on': r_on}
    else:
        reference_memristor_params = {'time_series_resolution': 1e-10,
                                  'r_off': memtorch.bh.StochasticParameter(loc=r_off, scale=sigma*2, min=1),
                                  'r_on': memtorch.bh.StochasticParameter(loc=r_on, scale=sigma, min=1)}

    patched_model = patch_model(copy.deepcopy(model_),
                              memristor_model=reference_memristor,
                              memristor_model_params=reference_memristor_params,
                              module_parameters_to_patch=[torch.nn.Linear, torch.nn.Conv2d],
                              mapping_routine=naive_map,
                              transistor=True,
                              programming_routine=None,
                              scheme=memtorch.bh.Scheme.DoubleColumn,
                              tile_shape=tile_shape,
                              max_input_voltage=0.3,
                              ADC_resolution=int(ADC_resolution),
                              ADC_overflow_rate=0.,
                              quant_method='linear')
    
    patched_model.tune_()
    return test(patched_model, test_loader)

df = pd.DataFrame(columns=['tile_shape', 'ADC_resolution', 'sigma', 'test_set_accuracy'])
tile_shape = [(256, 64)]
ADC_resolution = np.linspace(2, 10, num=5, endpoint=True, dtype=int)
sigma = np.logspace(6, 7, endpoint=True, num=5)
for tile_shape_ in tile_shape:
    for ADC_resolution_ in ADC_resolution:
        for sigma_ in sigma:
            print('tile_shape: %s; ADC_resolution: %d; sigma: %d' % (tile_shape_, ADC_resolution_, sigma_))
            df = df.append({'tile_shape': tile_shape_, 
                            'ADC_resolution': ADC_resolution_, 
                            'sigma': sigma_, 
                            'test_set_accuracy': trial(r_on, r_off, tile_shape_, ADC_resolution_, sigma_)}, ignore_index=True)
            df.to_csv('1_AE.csv', index=False)

## 4. Figure 1 [B,F]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import memtorch
from memtorch.utils import LoadCIFAR10
import numpy as np
import pandas as pd
from mobilenetv2 import MobileNetV2
import torchvision
import copy
from memtorch.mn.Module import patch_model
from memtorch.map.Parameter import naive_map
from memtorch.bh.crossbar.Program import naive_program
from memtorch.bh.nonideality.NonIdeality import apply_nonidealities


def test(model, test_loader):
    correct = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        output = model(data.to(device))
        pred = output.data.max(1)[1]
        correct += pred.eq(target.to(device).data.view_as(pred)).cpu().sum()

    return 100. * float(correct) / float(len(test_loader.dataset))

device = torch.device('cuda')
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=256, shuffle=False, num_workers=1)
model = MobileNetV2().to(device)
try:
    model.load_state_dict(torch.load('trained_model.pt'), strict=False)
    model.eval()
except:
    raise Exception('trained_model.pt has not been found.')

print('Test Set Accuracy: \t%2.2f%%' % test(model, test_loader))

model = MobileNetV2().to(device)
model.load_state_dict(torch.load('trained_model.pt'), strict=True)
model.eval()
print(test(model, test_loader))
r_on = 1.4e4
r_off = 5e7

def trial(r_on, r_off, tile_shape, ADC_resolution, conductance_states):
    model_ = copy.deepcopy(model)
    reference_memristor = memtorch.bh.memristor.VTEAM
    reference_memristor_params = {'time_series_resolution': 1e-10, 'r_off': r_off, 'r_on': r_on}
    patched_model = patch_model(copy.deepcopy(model_),
                              memristor_model=reference_memristor,
                              memristor_model_params=reference_memristor_params,
                              module_parameters_to_patch=[torch.nn.Linear, torch.nn.Conv2d],
                              mapping_routine=naive_map,
                              transistor=True,
                              programming_routine=None,
                              scheme=memtorch.bh.Scheme.DoubleColumn,
                              tile_shape=tile_shape,
                              max_input_voltage=0.3,
                              ADC_resolution=int(ADC_resolution),
                              ADC_overflow_rate=0.,
                              quant_method='linear')

    patched_model = apply_nonidealities(patched_model,
                                        non_idealities=[memtorch.bh.nonideality.NonIdeality.FiniteConductanceStates],
                                        conductance_states = int(conductance_states))
    
    patched_model.tune_()
    return test(patched_model, test_loader)

df = pd.DataFrame(columns=['tile_shape', 'ADC_resolution', 'conductance_states', 'test_set_accuracy'])
torch.backends.cudnn.benchmark = False
tile_shape = [(128, 128), (256, 64)]
ADC_resolution = np.linspace(2, 10, num=5, endpoint=True, dtype=int)
conductance_states = np.linspace(2, 10, num=5, endpoint=True, dtype=int)
for tile_shape_ in tile_shape:
    for ADC_resolution_ in ADC_resolution:
        for conductance_states_ in conductance_states:
            print('tile_shape: %s; ADC_resolution: %d; conductance_states: %d' % (tile_shape_, ADC_resolution_, conductance_states_))
            df = df.append({'tile_shape': tile_shape_, 
                            'ADC_resolution': ADC_resolution_, 
                            'conductance_states': conductance_states_, 
                            'test_set_accuracy': trial(r_on, r_off, tile_shape_, ADC_resolution_, conductance_states_)}, ignore_index=True)
            df.to_csv('1_BF.csv', index=False)

## 5. Figure 1 [C,G]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import memtorch
from memtorch.utils import LoadCIFAR10
import numpy as np
import pandas as pd
from mobilenetv2 import MobileNetV2
import torchvision
import copy
from memtorch.mn.Module import patch_model
from memtorch.map.Parameter import naive_map
from memtorch.bh.crossbar.Program import naive_program
from memtorch.bh.nonideality.NonIdeality import apply_nonidealities


def test(model, test_loader):
    correct = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        output = model(data.to(device))
        pred = output.data.max(1)[1]
        correct += pred.eq(target.to(device).data.view_as(pred)).cpu().sum()

    return 100. * float(correct) / float(len(test_loader.dataset))

device = torch.device('cuda')
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=256, shuffle=False, num_workers=1)
model = MobileNetV2().to(device)
try:
    model.load_state_dict(torch.load('trained_model.pt'), strict=False)
    model.eval()
except:
    raise Exception('trained_model.pt has not been found.')

print('Test Set Accuracy: \t%2.2f%%' % test(model, test_loader))

model = MobileNetV2().to(device)
model.load_state_dict(torch.load('trained_model.pt'), strict=True)
model.eval()
print(test(model, test_loader))
r_on = 1.4e4
r_off = 5e7

def trial(r_on, r_off, tile_shape, ADC_resolution, failure_percentage):
    model_ = copy.deepcopy(model)
    reference_memristor = memtorch.bh.memristor.VTEAM
    reference_memristor_params = {'time_series_resolution': 1e-10, 'r_off': r_off, 'r_on': r_on}
    patched_model = patch_model(copy.deepcopy(model_),
                              memristor_model=reference_memristor,
                              memristor_model_params=reference_memristor_params,
                              module_parameters_to_patch=[torch.nn.Linear, torch.nn.Conv2d],
                              mapping_routine=naive_map,
                              transistor=True,
                              programming_routine=None,
                              scheme=memtorch.bh.Scheme.DoubleColumn,
                              tile_shape=tile_shape,
                              max_input_voltage=0.3,
                              ADC_resolution=int(ADC_resolution),
                              ADC_overflow_rate=0.,
                              quant_method='linear')

    patched_model = apply_nonidealities(patched_model,
                                        non_idealities=[memtorch.bh.nonideality.NonIdeality.DeviceFaults],
                                        lrs_proportion=failure_percentage,
                                        hrs_proportion=0.,
                                        electroform_proportion=0.)
    
    patched_model.tune_()
    return test(patched_model, test_loader)

df = pd.DataFrame(columns=['tile_shape', 'ADC_resolution', 'failure_percentage', 'test_set_accuracy'])
tile_shape = [(128, 128), (256, 64)]
ADC_resolution = np.linspace(2, 10, num=5, endpoint=True, dtype=int)
failure_percentage = np.linspace(0, 0.25, 5)
for tile_shape_ in tile_shape:
    for ADC_resolution_ in ADC_resolution:
        for failure_percentage_ in failure_percentage:
            print('tile_shape: %s; ADC_resolution: %d; failure_percentage: %d' % (tile_shape_, ADC_resolution_, failure_percentage_))
            df = df.append({'tile_shape': tile_shape_, 
                            'ADC_resolution': ADC_resolution_, 
                            'failure_percentage': failure_percentage_, 
                            'test_set_accuracy': trial(r_on, r_off, tile_shape_, ADC_resolution_, failure_percentage_)}, ignore_index=True)
            df.to_csv('1_CG.csv', index=False)

## 6. Figure 1 [D,H]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import memtorch
from memtorch.utils import LoadCIFAR10
import numpy as np
import pandas as pd
from mobilenetv2 import MobileNetV2
import torchvision
import copy
from memtorch.mn.Module import patch_model
from memtorch.map.Parameter import naive_map
from memtorch.bh.crossbar.Program import naive_program
from memtorch.bh.nonideality.NonIdeality import apply_nonidealities


def test(model, test_loader):
    correct = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        output = model(data.to(device))
        pred = output.data.max(1)[1]
        correct += pred.eq(target.to(device).data.view_as(pred)).cpu().sum()

    return 100. * float(correct) / float(len(test_loader.dataset))

device = torch.device('cuda')
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=256, shuffle=False, num_workers=1)
model = MobileNetV2().to(device)
try:
    model.load_state_dict(torch.load('trained_model.pt'), strict=False)
    model.eval()
except:
    raise Exception('trained_model.pt has not been found.')

print('Test Set Accuracy: \t%2.2f%%' % test(model, test_loader))

model = MobileNetV2().to(device)
model.load_state_dict(torch.load('trained_model.pt'), strict=True)
model.eval()
print(test(model, test_loader))
r_on = 1.4e4
r_off = 5e7

def trial(r_on, r_off, tile_shape, ADC_resolution, failure_percentage):
    model_ = copy.deepcopy(model)
    reference_memristor = memtorch.bh.memristor.VTEAM
    reference_memristor_params = {'time_series_resolution': 1e-10, 'r_off': r_off, 'r_on': r_on}
    patched_model = patch_model(copy.deepcopy(model_),
                              memristor_model=reference_memristor,
                              memristor_model_params=reference_memristor_params,
                              module_parameters_to_patch=[torch.nn.Linear, torch.nn.Conv2d],
                              mapping_routine=naive_map,
                              transistor=True,
                              programming_routine=None,
                              scheme=memtorch.bh.Scheme.DoubleColumn,
                              tile_shape=tile_shape,
                              max_input_voltage=0.3,
                              ADC_resolution=int(ADC_resolution),
                              ADC_overflow_rate=0.,
                              quant_method='linear')

    patched_model = apply_nonidealities(patched_model,
                                        non_idealities=[memtorch.bh.nonideality.NonIdeality.DeviceFaults],
                                        lrs_proportion=0.,
                                        hrs_proportion=failure_percentage,
                                        electroform_proportion=0.)
    
    patched_model.tune_()
    return test(patched_model, test_loader)

df = pd.DataFrame(columns=['tile_shape', 'ADC_resolution', 'failure_percentage', 'test_set_accuracy'])
tile_shape = [(128, 128), (256, 64)]
ADC_resolution = np.linspace(2, 10, num=5, endpoint=True, dtype=int)
failure_percentage = np.linspace(0, 0.25, 5)
for tile_shape_ in tile_shape:
    for ADC_resolution_ in ADC_resolution:
        for failure_percentage_ in failure_percentage:
            print('tile_shape: %s; ADC_resolution: %d; failure_percentage: %d' % (tile_shape_, ADC_resolution_, failure_percentage_))
            df = df.append({'tile_shape': tile_shape_, 
                            'ADC_resolution': ADC_resolution_, 
                            'failure_percentage': failure_percentage_, 
                            'test_set_accuracy': trial(r_on, r_off, tile_shape_, ADC_resolution_, failure_percentage_)}, ignore_index=True)
            df.to_csv('1_DH.csv', index=False)